### General advice (delete this cell before submitting for review)

> * When adding **Products used**, embed the hyperlink to that specific product on the DE Africa Explorer using the `[product_name](product url)` syntax.
> * When writing in Markdown cells, start each sentence on a **new line**.
This makes it easy to see changes through git commits.
> * To faciliate the easy conversion of these notebooks into a docs help page, check the [known issues](https://github.com/GeoscienceAustralia/dea-docs/wiki/Known-issues) for formatting regarding the conversion of notebooks to DE Africa docs using Sphinx.
Things to be aware of:
    * Sphinx is highly sensitive to bulleted lists:
        * Ensure that there is an empty line between any preceding text and the list
        * Only use the `*` bullet (`-` is not recognised)
        * Sublists must be indented by 4 spaces
    * Two kinds of formatting cannot be used simultaneously:
        * Hyperlinked code: \[\`code_format\`](hyperlink) fails
        * Bolded code: \*\*\`code_format\`\*\* fails
    * Headers must appear in heirachical order (`#`, `##`, `###`, `####`) and there can only be one title (`#`).
> * Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
> * In the final notebook cell, include a set of relevant **keywords** which are used to build the DE Africa User Guide's [keyword Index](https://digital-earth-africa.readthedocs.io/en/latest/genindex.html).
    * Use the list of approved documentation keywords on the [wiki page](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/wiki/List-of-Documentation-Keywords).
    * Avoid using keywords that link to specific modules in `deafrica_tools`.
    * Use all lower-case (unless the tag is an acronym), separate words with spaces
    * Ensure the keywords cell below is in `Raw` format, rather than `Markdown` or `Code`.


# Sepctral-temporal feature inspection tool

* **Products used:** 
[ls8_sr](https://explorer.digitalearth.africa/ls8_sr), 
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a),
[s1_rtc](https://explorer.digitalearth.africa/s2_rtc),


## Background

This notebook is an outcome of the ODC 2021 women sprint. 

Spectral-temporal features from time-seris Earth observation data are often used to distinguish land cover types and monitor their evolution over time. They are particularly useful for understanding dynamic ecosystems such as wetlands.

Many methods have been developed to derive spectral and temporal features from optical or radar remote sensing data. Their effectiveness vary for different environments.
Therefore, we want to develop a tool that allows a scientist to quickly inspect well-known spectral indices and time series metrics for any location.


Contributors:

Fang Yuan, Bex Dunn, Meghan Halabisky, Kate Fickas, Allison Bailey, (add yourself here)




## Description

We want to build a tool that is:

* interactive
* using popular indicies and metrics
* easy to use for beginners wanting to learn more about remote sensing
* able to use all DE Africa input time series data
* expandable to include more spectral and temporal features.



***

## To-do list

**Completed:**
- polygon selector off map [ x ]
- sentinel 2 data [ x ]

**Must have:**
- polygon selector off map [ x ] 
- sentinel 2 data [ x ]
- all tasselled cap indices
- time series plot for each  
- a new function that lets you do the things, or all the functionality in this notebook.
    
**Would like to have:** _(links to where you can get code to do this)_
- multiple datasets (add landsat data) [using load ard notebook](./Using_load_ard.ipynb)
- multiple indices (for a list, see [bandindices.py](../Tools/deafrica_tools/bandindices.py) or the [band index notebook](Calculating_band_indices.ipynb))
- sentinel 1 data [sentinel1 notebook](../Datasets/Sentinel_1.ipynb)
- spectral plot for each
- climate data [era 5 climate data notebook](../Datasets/Climate_Data_ERA5_AWS.ipynb )
- soil moisture [soil moisture notebook](../Datasets/Soil_Moisture.ipynb) 
- evapotranspiration
- phenology/metrics/cycles [Vegetation_phenology.ipynb](./Vegetation_phenology.ipynb)
- click on plotted points to retrieve the imagery for that timestamp
        

## Getting started

Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

 - to start with we're going to munge in some code from the `Crop_health.ipynb` notebook in the `Real_world_examples` folder

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [1]:
%matplotlib inline

import datacube
import numpy as np
import sys
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

#### Work in progress - this is the crop health app to take to bits and change -

In [2]:
# notebookapp_spectraltemporal.py
'''
This file contains functions for loading and interacting with data in the
spectral temporal widget notebook in the Frequently used code folder.

Available functions:
    load_crophealth_data
    run_crophelath_app

Last modified: April 2021
'''

# Load modules
from ipyleaflet import (
    Map,
    GeoJSON,
    DrawControl,
    basemaps
)
import datetime as dt
import datacube
from osgeo import ogr
import matplotlib as mpl
import matplotlib.pyplot as plt
import rasterio
from rasterio.features import geometry_mask
import xarray as xr
from IPython.display import display
import warnings
import ipywidgets as widgets
import json
import geopandas as gpd
from io import BytesIO

# Load utility functions
from deafrica_tools.datahandling import load_ard
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.bandindices import calculate_indices


def load_crophealth_data(lat, lon, buffer):
    """
    Loads Sentinel-2 analysis-ready data (ARD) product for the crop health
    case-study area over the last two years.
    Last modified: April 2020
    
    Parameters
    ----------
    lat: float
        The central latitude to analyse
    lon: float
        The central longitude to analyse
    buffer:
         The number of square degrees to load around the central latitude and longitude. 
         For reasonable loading times, set this as `0.1` or lower.

    Returns
    ----------
    ds: xarray.Dataset 
        data set containing combined, masked data
        Masked values are set to 'nan'
    """
    
    # Suppress warnings
    warnings.filterwarnings('ignore')

    # Initialise the data cube. 'app' argument is used to identify this app
    dc = datacube.Datacube(app='Crophealth-app')
    
    # Define area to load
    latitude = (lat - buffer, lat + buffer)
    longitude = (lon - buffer, lon + buffer)

    # Specify the date range
    # Calculated as today's date, subtract 730 days to collect two years of data
    # Dates are converted to strings as required by loading function below
    end_date = dt.date.today()
    start_date = end_date - dt.timedelta(days=730)

    time = (start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    # Construct the data cube query
    products = ["s2_l2a"]
    
    query = {
        'x': longitude,
        'y': latitude,
        'time': time,
                'measurements': [
            'red',
            'green',
            'blue',
            'nir',
            'swir_2'
        ],
        'output_crs': 'EPSG:6933',
        'resolution': (-20, 20)
    }

    # Load the data and mask out bad quality pixels
    ds = load_ard(dc, products=products, min_gooddata=0.5, **query)

    # Calculate the normalised difference vegetation index (NDVI) across
    # all pixels for each image.
    # This is stored as an attribute of the data
    ds = calculate_indices(ds, index='NDVI', collection='s2')

    # Return the data
    return(ds)


def run_crophealth_app(ds, lat, lon, buffer):
    """
    Plots an interactive map of the crop health case-study area and allows
    the user to draw polygons. This returns a plot of the average NDVI value
    in the polygon area.
    Last modified: January 2020
    
    Parameters
    ----------
    ds: xarray.Dataset 
        data set containing combined, masked data
        Masked values are set to 'nan'
    lat: float
        The central latitude corresponding to the area of loaded ds
    lon: float
        The central longitude corresponding to the area of loaded ds
    buffer:
         The number of square degrees to load around the central latitude and longitude. 
         For reasonable loading times, set this as `0.1` or lower.
    """
    
    # Suppress warnings
    warnings.filterwarnings('ignore')

    # Update plotting functionality through rcParams
    mpl.rcParams.update({'figure.autolayout': True})
    
    # Define polygon bounds   
    latitude = (lat - buffer, lat + buffer)
    longitude = (lon - buffer, lon + buffer)

    # Define the bounding box that will be overlayed on the interactive map
    # The bounds are hard-coded to match those from the loaded data
    geom_obj = {
        "type": "Feature",
        "properties": {
            "style": {
                "stroke": True,
                "color": 'red',
                "weight": 4,
                "opacity": 0.8,
                "fill": True,
                "fillColor": False,
                "fillOpacity": 0,
                "showArea": True,
                "clickable": True
            }
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                    [
                        longitude[0],
                        latitude[0]
                    ],
                    [
                        longitude[1],
                        latitude[0]
                    ],
                    [
                        longitude[1],
                        latitude[1]
                    ],
                    [
                        longitude[0],
                        latitude[1]
                    ],
                    [
                        longitude[0],
                        latitude[0]
                    ]
                ]
            ]
        }
    }
    
    # Create a map geometry from the geom_obj dictionary
    # center specifies where the background map view should focus on
    # zoom specifies how zoomed in the background map should be
    loadeddata_geometry = ogr.CreateGeometryFromJson(str(geom_obj['geometry']))
    loadeddata_center = [
        loadeddata_geometry.Centroid().GetY(),
        loadeddata_geometry.Centroid().GetX()
    ]
    loadeddata_zoom = 16

    # define the study area map
    studyarea_map = Map(
        center=loadeddata_center,
        zoom=loadeddata_zoom,
        basemap=basemaps.Esri.WorldImagery
    )

    # define the drawing controls
    studyarea_drawctrl = DrawControl(
        polygon={"shapeOptions": {"fillOpacity": 0}},
        marker={},
        circle={},
        circlemarker={},
        polyline={},
    )

    # add drawing controls and data bound geometry to the map
    studyarea_map.add_control(studyarea_drawctrl)
    studyarea_map.add_layer(GeoJSON(data=geom_obj))

    # Index to count drawn polygons
    polygon_number = 0

    # Define widgets to interact with
    instruction = widgets.Output(layout={'border': '1px solid black'})
    with instruction:
        print("Draw a polygon within the red box to view a plot of "
              "average NDVI over time in that area.")

    info = widgets.Output(layout={'border': '1px solid black'})
    with info:
        print("Plot status:")

    fig_display = widgets.Output(layout=widgets.Layout(
        width="50%",  # proportion of horizontal space taken by plot
    ))

    with fig_display:
        plt.ioff()
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.set_ylim([0, 1])

    colour_list = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Function to execute each time something is drawn on the map
    def handle_draw(self, action, geo_json):
        nonlocal polygon_number

        # Execute behaviour based on what the user draws
        if geo_json['geometry']['type'] == 'Polygon':

            info.clear_output(wait=True)  # wait=True reduces flicker effect
            
            # Save geojson polygon to io temporary file to be rasterized later
            jsonData = json.dumps(geo_json)
            binaryData = jsonData.encode()
            io = BytesIO(binaryData)
            io.seek(0)
            
            # Read the polygon as a geopandas dataframe
            gdf = gpd.read_file(io)
            gdf.crs = "EPSG:4326"

            # Convert the drawn geometry to pixel coordinates
            xr_poly = xr_rasterize(gdf, ds.NDVI.isel(time=0), crs='EPSG:6933')

            # Construct a mask to only select pixels within the drawn polygon
            masked_ds = ds.NDVI.where(xr_poly)
            
            masked_ds_mean = masked_ds.mean(dim=['x', 'y'], skipna=True)
            colour = colour_list[polygon_number % len(colour_list)]

            # Add a layer to the map to make the most recently drawn polygon
            # the same colour as the line on the plot
            studyarea_map.add_layer(
                GeoJSON(
                    data=geo_json,
                    style={
                        'color': colour,
                        'opacity': 1,
                        'weight': 4.5,
                        'fillOpacity': 0.0
                    }
                )
            )

            # add new data to the plot
            xr.plot.plot(
                masked_ds_mean,
                marker='*',
                color=colour,
                ax=ax
            )

            # reset titles back to custom
            ax.set_title("Average NDVI from Sentinel-2")
            ax.set_xlabel("Date")
            ax.set_ylabel("NDVI")

            # refresh display
            fig_display.clear_output(wait=True)  # wait=True reduces flicker effect
            with fig_display:
                display(fig)
                
            with info:
                print("Plot status: polygon sucessfully added to plot.")

            # Iterate the polygon number before drawing another polygon
            polygon_number = polygon_number + 1

        else:
            info.clear_output(wait=True)
            with info:
                print("Plot status: this drawing tool is not currently "
                      "supported. Please use the polygon tool.")

    # call to say activate handle_draw function on draw
    studyarea_drawctrl.on_draw(handle_draw)

    with fig_display:
        # TODO: update with user friendly something
        display(widgets.HTML(""))

    # Construct UI:
    #  +-----------------------+
    #  | instruction           |
    #  +-----------+-----------+
    #  |  map      |  plot     |
    #  |           |           |
    #  +-----------+-----------+
    #  | info                  |
    #  +-----------------------+
    ui = widgets.VBox([instruction,
                       widgets.HBox([studyarea_map, fig_display]),
                       info])
    display(ui)


/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
#---from the crop health notebook - update these with a less specific set once we rewrite them ---#
sys.path.append("../Scripts")
from notebookapp_crophealth import load_crophealth_data
from notebookapp_crophealth import run_crophealth_app

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [4]:
dc = datacube.Datacube(app='Spectral_temporal_widget')

/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


### Analysis parameters

The following cell sets important parameters for the analysis. 
There are three parameters that control where the data will be loaded:

* `lat`: The central latitude to analyse (e.g. `-19.3`).
* `lon`: The central longitude to analyse (e.g. `23.3`).
* `buffer`: The number of square degrees to load around the central latitude and longitude. For reasonable loading times, set this as `0.1` or lower.

These can be changed in the cell below, noting that the [DE Africa Explorer](https://explorer.digitalearth.africa/ls8_usgs_sr_scene) can be used to check whether data is available over the selected area.


In [5]:
# Define the area of interest for the analysis
lat = -19.3
lon = 23.3
buffer = 0.005

## Load some data
The `load_crophealth_data()` command performs several key steps:

* identify all available Sentinel-2 data in the case-study area over the last two years
* remove any bad quality pixels
* keep images where more than half of the image contains good quality pixels
* calculate the NDVI from the red and near infrared bands
* return the collated data for analysis

The cleaned and collated data is stored in the `dataset` object.
As the command runs, feedback will be provided below the cell, including information on the number of cleaned images loaded from the satellite.

The function takes three arguments: `lat`, `lon`, and `buffer`.
These determine the area of interest that the function loads, and can be changed in the previous cell.

**Please be patient**.
The load is complete when the cell status goes from `[*]` to `[number]`.

In [6]:
# load the data based off our set parameters
dataset = load_crophealth_data(lat, lon, buffer)

Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 110 out of 145 time steps with at least 50.0% good quality pixels
Applying pixel quality/cloud mask
Loading 110 time steps


## Run the crop health app
The `run_crophealth_app()` command launches an interactive map.
Drawing polygons within the red boundary (which represents the area covered by the loaded data) will result in plots of the average NDVI in that area.
Draw polygons by clicking the &#11039; symbol in the app.

The function works by taking the loaded data `dataset` as an argument, as well as the `lat`, `lon`, and `buffer` parameters used to define the spatial extent.

> **Note:** data points will only appear for images where more than 50% of the pixels were classified as good quality. This may cause trend lines on the average NDVI plot to appear disconnected. Available data points will be marked with the `*` symbol.

In [7]:
# run the app
run_crophealth_app(dataset, lat, lon, buffer)

## Ok! It's alive! now we want to set this up for wetlandy stuff
Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [8]:
# Use code comments for low-level documentation of code
c = 3

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [9]:
print(datacube.__version__)

1.8.4.dev81+g80d466a2


**Last Tested:**

In [10]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2021-06-23'